In [1]:
from Kompas import Kompas #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [21]:
# health
obj = Kompas()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 1)
init_page = 1
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['ekonomi','ekonomi'],['bola','bola'],
            ['tekno','tekno'],['sains','sains'],['entertainment','entertainment'],['lifestyle','lifestyle'],
            ['travel','travel'],['edukasi','edukasi']]
init_cat = list_cat[9:]

In [26]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y-%m-%d"))
        init_page = 1
    init_cat = list_cat

2018-01-01
travel
page  1
https://indeks.kompas.com/travel/2018-01-01/1
Insert berita  Tahun Baru Usai, Yuk Intip Tradisi Barbekyu di Belahan Dunia
salah2
Insert berita  Kisah Penjual Sate Ayam yang Diajak Jokowi Berjualan di Gedung Agung
salah2
Insert berita  Ini Cara Taman Bunga Nusantara Antisipasi Libur Tahun Baru 2018
salah2
Insert berita  Simak 7 Tips Melancong dari Pramugari
salah2
Insert berita  Sensasi Menginap di Istana Raja Kotawaringin...
salah2
Insert berita  Tahun Baru, 10 Museum dan Tempat Wisata Ini Tetap Buka
salah2


TypeError: 'NoneType' object is not subscriptable